In [1]:
from DataLoader import DataLoader
from DataSetEnum import DataSet

In [2]:
val_df = DataLoader(DataSet.VALIDATION).df
test_df = DataLoader(DataSet.TEST).df

24/03/13 10:31:10 WARN Utils: Your hostname, Juliuss-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 172.18.14.183 instead (on interface en0)
24/03/13 10:31:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 10:31:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/13 10:31:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/d

Removed 0 duplicate rows.
CodeCache: size=131072Kb used=21899Kb max_used=21911Kb free=109172Kb
 bounds [0x000000010b9e0000, 0x000000010cf60000, 0x00000001139e0000]
 total_blobs=8606 nmethods=7668 adapters=849
 compilation: disabled (not enough contiguous free space left)
Removed 0 duplicate rows.


24/03/13 10:31:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/test_hidden.csv
Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


In [3]:
from pyspark.sql.functions import lit

# tried every fucking possible data type for true and false, 
# but when submitting predictions, the website always complains that
# "predictions can only be 'True' or 'False"..."
val_df = val_df.withColumn("label", lit(True).cast("string"))
# set datatype to string

test_df = test_df.withColumn("label", lit(False).cast("string"))



In [4]:
# check if all avlues of label column are 'true'
val_df.groupBy("label").count().show()
test_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
| true| 1248|
+-----+-----+

+-----+-----+
|label|count|
+-----+-----+
|false| 1136|
+-----+-----+



In [5]:
# val_df.show()

In [6]:
val_df.write.csv(f'predictions/validation.csv', mode="overwrite", header=False, index=False)
test_df.write.csv(f'predictions/test.csv', mode="overwrite", header=False,

TypeError: DataFrameWriter.csv() got an unexpected keyword argument 'index'

In [ ]:
# export to .csv
# df.write.csv("predictions/validation.csv", header=True)

In [ ]:
# add column label
